# Carga del archivo y preprocesamiento

In [2]:
import pandas as pd

# Cargar el archivo con la descripción - Remoto
#arco = "C:/Backup - Archivos de trabajo/DNP/Automatización ArCo/Excel/2024 11 01 - Base Original.xlsx"


# Cargar el archivo con la descripción - Remoto
arco = "C:/Users/supegui/Documents/Metodología ArCo/Python/Excel/2024 11 01 - Base Original.xlsx"

df_palabras_clave = pd.read_excel(arco)

#df_palabras_clave

descripcion = df_palabras_clave['Descripción']
#Descripción
df_palabras_clave['Descripción'] = df_palabras_clave['Descripción'].astype(str)


In [4]:
pip install --upgrade spacy

Note: you may need to restart the kernel to use updated packages.
   ---------------------------------------- 0.0/12.3 MB ? eta -:--:--
   -- ------------------------------------- 0.8/12.3 MB 4.2 MB/s eta 0:00:03
   ---- ----------------------------------- 1.3/12.3 MB 3.9 MB/s eta 0:00:03
   ----- ---------------------------------- 1.8/12.3 MB 2.8 MB/s eta 0:00:04
   ----------- ---------------------------- 3.4/12.3 MB 4.1 MB/s eta 0:00:03
   --------------- ------------------------ 4.7/12.3 MB 4.5 MB/s eta 0:00:02
   -------------------- ------------------- 6.3/12.3 MB 5.1 MB/s eta 0:00:02
   ----------------------- ---------------- 7.3/12.3 MB 5.0 MB/s eta 0:00:01
   --------------------------- ------------ 8.4/12.3 MB 5.0 MB/s eta 0:00:01
   ------------------------------ --------- 9.4/12.3 MB 5.3 MB/s eta 0:00:01
   ---------------------------------- ----- 10.5/12.3 MB 5.1 MB/s eta 0:00:01
   -------------------------------------- - 11.8/12.3 MB 5.1 MB/s eta 0:00:01
   ------------

  You can safely remove it manually.


In [6]:
!python -m spacy download es_core_news_sm

     ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
     ----- ---------------------------------- 1.8/12.9 MB 8.4 MB/s eta 0:00:02
     --------- ------------------------------ 3.1/12.9 MB 8.0 MB/s eta 0:00:02
     ----------- ---------------------------- 3.7/12.9 MB 5.7 MB/s eta 0:00:02
     --------------- ------------------------ 5.0/12.9 MB 5.7 MB/s eta 0:00:02
     --------------------- ------------------ 7.1/12.9 MB 6.5 MB/s eta 0:00:01
     --------------------------- ------------ 8.9/12.9 MB 6.8 MB/s eta 0:00:01
     ---------------------------- ----------- 9.2/12.9 MB 6.1 MB/s eta 0:00:01
     --------------------------------- ------ 10.7/12.9 MB 6.3 MB/s eta 0:00:01
     ---------------------------------------  12.8/12.9 MB 6.7 MB/s eta 0:00:01
     ---------------------------------------- 12.9/12.9 MB 6.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


# Validación y exportación del resultado

In [3]:
import pandas as pd
import spacy

# Cargar el modelo de lenguaje de spaCy
nlp = spacy.load("es_core_news_sm")


criterios = {
    # Investigación
    "TRL1": ["Investigación básica", "descubrimiento", "principios fundamentales", "exploración científica", "teoría", "análisis preliminar"],
    "TRL2": ["formulación conceptual", "hipótesis", "diseño conceptual", "modelo teórico", "simulación"],
    "TRL3": ["validación experimental", "prueba experimental", "prueba de concepto", "experimentación", "ensayo inicial", "prototipo conceptual", "viabilidad técnica"],
    # Desarrollo
    "TRL4": ["validación en laboratorio", "entorno controlado", "pruebas de laboratorio", "simulación técnica", "prototipo"],
    "TRL5": ["entorno relevante", "condiciones simuladas", "pruebas funcionales", "análisis de desempeño"],
    "TRL6": ["prototipo funcional", "entorno operacional simulado", "demostración práctica", "validación aplicada", "tecnología precomercial", "pruebas significativas"],
    # Innovación
    "TRL7": ["pruebas en condiciones reales", "prototipo operativo", "entorno final", "validación del sistema", "simulación industrial"],
    "TRL8": ["sistema completo", "tecnología lista", "certificación", "calificación final", "integración total", " operación piloto"],
    "TRL9": ["implementación comercial", "pruebas finales", "operación real", "producto terminado", "sistema operacional"]
}


# Crear un DataFrame vacío con las columnas de criterios
df_validaciones = pd.DataFrame(columns=criterios.keys())

# Función para verificar si una descripción contiene un criterio
def contiene_criterio(descripcion, criterio):
    doc = nlp(descripcion.lower())
    #return criterio.lower() in [token.text for token in doc]
    return any(keyword.strip().lower() in doc.text for keyword in criterios[criterio])


# Iterar a través de las descripciones y los criterios y agregar las validaciones a df_validaciones
for index, row in df_palabras_clave.iterrows():
    descripcion = row['Descripción']
    validaciones = {}
    for criterio in criterios:
        validaciones[criterio] = 'Sí' if contiene_criterio(descripcion, criterio) else 'No'
    #df_validaciones = df_validaciones.append(validaciones, ignore_index=True)#
    df_validaciones = pd.concat([df_validaciones, pd.DataFrame(validaciones, index=[0])], ignore_index=True)

# Añadir columna "Otra" al DataFrame de validaciones
df_validaciones["Otra"] = df_validaciones.apply(
    lambda row: "Sí" if all(value == "No" for value in row.values) else "No", axis=1
)

# Concatenar las descripciones originales con el DataFrame de validaciones
df_resultado = pd.concat([df_palabras_clave, df_validaciones], axis=1)

# Nombre del archivo Excel de salida - Remoto
#archivo_salida = "C:/Backup - Archivos de trabajo/DNP/Automatización ArCo/Excel/2024 12 08 - validaciones_resultado.xlsx"

# Nombre del archivo Excel de salida - DNP
archivo_salida = "C:/Users/supegui/Documents/Metodología ArCo/Python/Excel/2024 12 09 - validaciones_resultado_trl.xlsx"

# Exportar el DataFrame con las validaciones a un archivo Excel
df_resultado.to_excel(archivo_salida, index=False)

print(f"Validaciones exportadas a {archivo_salida}")


Validaciones exportadas a C:/Users/supegui/Documents/Metodología ArCo/Python/Excel/2024 12 09 - validaciones_resultado_trl.xlsx
